# Behavior Trees

Behavior Trees are extremally powerfull ways to program/ explain robots behavior and its priorities

In [ ]:
!pip install py_trees

In [1]:
import rclpy
from rclpy.node import Node

from std_msgs.msg import String


class MinimalNode(Node):
    def __init__(self):
        super().__init__("service_node")


try:
    rclpy.init()

    minimal_node = MinimalNode()
except RuntimeError:
    pass

In [5]:
import py_trees
import random
from geometry_msgs.msg import PoseStamped
from nav2_simple_commander.robot_navigator import BasicNavigator, TaskResult
import rclpy
from rclpy.duration import Duration


class GoToPoint(py_trees.behaviour.Behaviour):
    def __init__(self, name, navigator):
        """
        Initializes the GoToPoint behavior with the given name and navigator.
        
        Args:
            name (str): The name of the behavior.
            navigator (BasicNavigator): The navigator used to control the robot.
        """
        self.navigator = navigator
        super(GoToPoint, self).__init__(name)

    def setup(self):
        """
        Sets up the necessary components for the behavior.
        
        This method is called to perform one-time initialization that is necessary
        before the behavior can run. This includes setting up hardware, initializing
        middleware (such as ROS publishers/subscribers), and any other setup tasks
        that might interfere with offline rendering or validation of the behavior.
        """
        goal_pose = PoseStamped()
        goal_pose.header.frame_id = "map"
        goal_pose.header.stamp = self.navigator.get_clock().now().to_msg()
        goal_pose.pose.position.x = 0.5
        goal_pose.pose.position.y = -1.2
        goal_pose.pose.orientation.w = 1.0
        
        self.goal_pose = goal_pose
        self.logger.debug("  %s [GoToPoint::setup()]" % self.name)

    def initialise(self):
        """
        Initializes the behavior before execution.
        
        This method is called the first time the behavior is ticked and anytime
        the status is not RUNNING thereafter. It prepares the behavior for execution,
        such as sending the goal pose to the navigator.
        """
        self.navigator.goToPose(self.goal_pose)
        self.logger.debug("  %s [GoToPoint::initialise()]" % self.name)

    def update(self):
        """
        Updates the behavior's state.
        
        This method is called every time the behavior is ticked. It monitors the
        status of the navigation task and returns the appropriate status:
        RUNNING, SUCCESS, or FAILURE.
        
        Returns:
            py_trees.common.Status: The status of the behavior.
        """
        if not self.navigator.isTaskComplete():
            feedback = self.navigator.getFeedback()
            print(feedback)
            return py_trees.common.Status.RUNNING
        
        result = self.navigator.getResult()

        if result == TaskResult.SUCCEEDED:
            self.feedback_message = "Navigation succeeded!"
            return py_trees.common.Status.SUCCESS
        else:
            self.feedback_message = "Navigation failed"
            return py_trees.common.Status.FAILURE

    def terminate(self, new_status):
        """
        Terminates the behavior.
        
        This method is called whenever the behavior switches to a non-running state.
        It can perform cleanup tasks or handle state transitions.
        
        Args:
            new_status (py_trees.common.Status): The new status of the behavior.
        """
        self.logger.debug("  %s [GoToPoint::terminate()][%s->%s]" % (self.name, self.status, new_status))



In [4]:
from geometry_msgs.msg import PoseStamped
from nav2_simple_commander.robot_navigator import BasicNavigator, TaskResult
import rclpy
from rclpy.duration import Duration

"""
Basic navigation demo to go to pose.
"""


navigator = BasicNavigator()
navigator.waitUntilNav2Active()


[INFO] [1717857413.712779188] [basic_navigator]: Nav2 is ready for use!


In [6]:
go_to_point_behavior = GoToPoint("Navigate to Point", navigator)


In [ ]:
if __name__ == '__main__':
    # Perform one-time setup for the behavior
    go_to_point_behavior.setup()

    # Create a behavior tree with the GoToPoint behavior as the root
    behaviour_tree = py_trees.trees.BehaviourTree(
        root=go_to_point_behavior
    )

    # Print the tree structure to the console
    print(py_trees.display.unicode_tree(root=go_to_point_behavior))

    # Setup the behavior tree, which includes setting up all child behaviors
    behaviour_tree.setup(timeout=15)

    def print_tree(tree):
        """
        A callback function to print the behavior tree structure
        with the status of each node after each tick.
        
        Args:
            tree (py_trees.trees.BehaviourTree): The behavior tree to print.
        """
        print(py_trees.display.unicode_tree(root=tree.root, show_status=True))

    try:
        # Start the behavior tree ticking, with a tick period of 500ms
        # and continuously ticking until interrupted
        behaviour_tree.tick_tock(
            period_ms=500,
            number_of_iterations=py_trees.trees.CONTINUOUS_TICK_TOCK,
            pre_tick_handler=None,
            post_tick_handler=print_tree  # Print the tree structure after each tick
        )
    except KeyboardInterrupt:
        # Gracefully handle shutdown when the user interrupts the program
        behaviour_tree.interrupt()

--> Navigate to Point



[INFO] [1717857438.615434057] [basic_navigator]: Navigating to goal: 0.5 -1.2...


nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=108, nanosec=567000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.03734669401243318, y=0.012764282553025641, z=0.009982607060399476), orientation=geometry_msgs.msg.Quaternion(x=1.926306708252327e-06, y=3.920810551312223e-06, z=0.0045319121979536395, w=0.9999897308236453))), navigation_time=builtin_interfaces.msg.Duration(sec=0, nanosec=14000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=0, distance_remaining=1.3604018688201904)
--> Navigate to Point [*]

nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=108, nanosec=771000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.05142725497886702, y=0.0128123065789917

nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=109, nanosec=587000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.21241622763638834, y=-0.053631417955921856, z=0.01107745357978231), orientation=geometry_msgs.msg.Quaternion(x=0.0022511614058646563, y=-2.476431018e-05, z=-0.35624746246149946, w=0.9343889324841107))), navigation_time=builtin_interfaces.msg.Duration(sec=1, nanosec=10000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=5, nanosec=35005687), number_of_recoveries=0, distance_remaining=1.1989949941635132)
--> Navigate to Point [*]

nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=109, nanosec=655000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.22536718186278204, y=-0.06580118196483

nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=110, nanosec=845000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.34400940812839753, y=-0.3338092168675205, z=0.011656732462225306), orientation=geometry_msgs.msg.Quaternion(x=0.0013209803799153056, y=-0.0005446189106361564, z=-0.6048526056602056, w=0.7963361625765453))), navigation_time=builtin_interfaces.msg.Duration(sec=2, nanosec=274000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=3, nanosec=643142315), number_of_recoveries=0, distance_remaining=0.9195480942726135)
--> Navigate to Point [*]

nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=110, nanosec=913000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.33942014551610633, y=-0.34920554

...

[INFO] [1717860602.252777301] [basic_navigator]: Navigating to goal: 0.5 -1.2...


--> Navigate to Point [✓] -- Navigation succeeded!



[INFO] [1717860604.234603206] [basic_navigator]: Navigating to goal: 0.5 -1.2...


--> Navigate to Point [✓] -- Navigation succeeded!



[INFO] [1717860606.835497790] [basic_navigator]: Navigating to goal: 0.5 -1.2...


# More convenient using py_trees_ros

[https://py-trees-ros-tutorials.readthedocs.io/en/devel/tutorials.html#the-mock-robot](https://py-trees-ros-tutorials.readthedocs.io/en/devel/tutorials.html#the-mock-robot)